In [239]:
import gensim
import pymystem3

In [240]:
# Функция возвращает список стихов с названиями
def parse(text):
    verse = ''
    verses = []
    for line in text:
        if line.startswith('МЕТКА'):
            continue
        elif line.startswith('НАЗВАНИЕ'):
            title = line.replace('НАЗВАНИЕ', '').strip()
            if not title.isnumeric():
                verse += title + '\n\n'
        elif line == 'ТЕКСТ\n':
            continue
        elif line == 'КОН\n':
            verses.append(verse)
            verse = ''
        else:
            verse += line
    
    return verses

In [318]:
# Функция преобразует список стихов в список списков токенов вида лемма_ЧАСТЬРЕЧИ
# Классификация частей речи в формате Mystem
# Возможен учёт стоп-слов
def preprocess(texts, stop_words=None):
    mystem = pymystem3.Mystem()
    preprocessed_texts = []
    
    for text in texts:
        preprocessed_text = []
        analized = mystem.analyze(text)
        
        for result in analized:
            if 'analysis' not in result:
                continue
            if result['analysis'] == []:
                continue
            lemma = result['analysis'][0]['lex']
            if stop_words is not None:
                if lemma in stop_words:
                    continue
            pos = result['analysis'][0]['gr'].split(',')[0].split('=')[0]
            preprocessed_text.append(lemma.lower() + '_' + pos)
            
        preprocessed_texts.append(preprocessed_text)
        
    return preprocessed_texts

In [319]:
# Создаёт и возвращает обученную модель векторизованных токенов
def create_model(preprocessed_texts, size=150, window=15, min_count=1, epochs=15):
    model = gensim.models.Word2Vec(preprocessed_texts,
                                   size=size, window=window, min_count=min_count)
    model.train(preprocessed_texts, total_examples=model.corpus_count,
                epochs=epochs)
    return model

In [391]:
# Возвращает список текстов из texts, которые удовлетворяют запросу по слову word
# topn - количество "похожих" слов, также используемых при поиске
# log - показать список слов, использвуемых при поиске (слово + похожие)
def search(word, model, texts, preprocessed_texts, topn=10, log=False):
    mystem = pymystem3.Mystem()
    analyzed = mystem.analyze(word)
    try:
        if 'analysis' not in analyzed[0]:
            raise KeyError('Incorrect search queue')
        if analyzed[0]['analysis'] == []:
            raise KeyError('Incorrect word')
        lemma = analyzed[0]['analysis'][0]['lex']
        pos = analyzed[0]['analysis'][0]['gr'].split(',')[0].split('=')[0]
        word = lemma + '_' + pos
        if log:
            print('Ищем: ' + word + ' ...')

        searched_texts = set()
        for i in range(len(preprocessed_texts)):
            if word in preprocessed_texts[i]:
                searched_texts.add(texts[i])
        if topn > 0:
            if log:
                print('А также:')
            for result in model.wv.most_similar(word, topn=topn):
                similar_word = result[0]
                if log:
                    print(similar_word)
                for i in range(len(preprocessed_texts)):
                    if similar_word in preprocessed_texts[i]:
                        searched_texts.add(texts[i])
            if log:
                print()
        return list(searched_texts)
    except KeyError:
        return ['Поиск не дал результатов']

In [321]:
# При препроцессировании можно учесть стоп-слова, хотя заумь лучше оставить как есть
f = open('stop_words.txt', 'r')
stop_words = []
for line in f:
    stop_words.append(line.strip())

In [384]:
# Обучим модель на сборнике Миши, не беря последние 3 стихотворения
verses = parse(open('taskO1_sokolov.txt', 'r').readlines())[:-3]
preprocessed_verses = preprocess(verses)
model = create_model(preprocessed_verses)

In [389]:
# Как выглядит словарь нашей модели
list(model.wv.vocab)[:10]

['что_SPRO',
 'это_PART',
 'так_ADVPRO',
 'красный_A',
 'рот_S',
 'у_PR',
 'жаба_S',
 'не_PART',
 'жевать_V',
 'ль_PART']

In [402]:
# Попробуем поискать знаменитое слово
# Пока что поиск может искать только слова!
for result in search('ко', model, verses, preprocessed_verses, topn=6, log=True):
    print(result + '\n')

Ищем: ко_PR ...
А также:
романюк_S
анечка_S
жека_S
подушка_S
комета_S
кобзарь_S

No one cares

Кокетничая запонками
из свеже-отравленных скорпионов
Портовый кран
вдвое вытянул
изумрудный перископ головы
и прикрыл
индиговым сатином
жабры,
дразня пролетающих с Олимпа
алебастровых богинь
цин-ко-но-жек!..


ОН

кобзарь
озарь
грандиозарь
виртуозарь


Анечке Романюк

ко ко
ко ко ко
ко ко
ко ко ко
ко ко
ко ко ко
ко
ко
ко ко
ко ко ко
ко
ко ко ко
ко
ко ко ко
ко
ко ко ко
ко ко
ко ко
ко ко
ко ко


No one cares

КОМЕТА ЗАБИЛАСЬ ко мне ПОД ПОДУШКУ
Жужжит и щекочет, целуя колючее ушко


No one cares

СМЕРТЬ ХУДОЖНИКА
привыкнув ко всем безобразьям
искал я их днем с фонарем
но увы! все износились проказы
не забыться мне ни на чем!
и взор устремивши к бесплотным
я тихо но твердо сказал:
мир вовсе не рвотное —
и мордой уткнулся в Обводный канал…




In [393]:
# Попробуем поискать слово, которое содержится в одном из последних трёх стихотворений
for result in search('слизняк', model, verses, preprocessed_verses, log=True):
    print(result + '\n')

Ищем: слизняк_S ...
А также:
Поиск не дал результатов



In [394]:
# Дообучает модель новыми списками токенов
def update_model(model, preprocessed_texts, epochs=15):
    model.build_vocab(preprocessed_texts, update=True)
    model.train(preprocessed_texts, total_examples=model.corpus_count,
                epochs=epochs)

In [395]:
# Дообучим нашу модель
new_verses = parse(open('taskO1_sokolov.txt', 'r').readlines())[-3:]
preprocessed_new_verses = preprocess(new_verses)
update_model(model, preprocessed_new_verses)

In [396]:
# Проверим словарь
list(model.wv.vocab)[:10]

['что_SPRO',
 'это_PART',
 'так_ADVPRO',
 'красный_A',
 'рот_S',
 'у_PR',
 'жаба_S',
 'не_PART',
 'жевать_V',
 'ль_PART']

In [397]:
# Проверим, что новое слово теперь есть в словаре
'слизняк_S' in model.wv.vocab

True

In [406]:
# Теперь он должен уметь искать новое слово и сам текст, в котором оно содержится
for result in search('слизняк', model, verses + new_verses,
                     preprocessed_verses + preprocessed_new_verses, topn=1, log=True):
    print(result + '\n')

Ищем: слизняк_S ...
А также:
мой_APRO

No one cares

В полночь я заметил на всоей простыне черного и
твердого,
величиной с клопа
в красной бахроме ножек.
Прижег его спичкой. А он, потолстел без ожога, как
повернутая дном железная бутылка…
Я подумал: мало огня?…
Но ведь для такого — спичка как бревно!…
Пришедшие мои друзья набросали на него щепок,
бумаги с керосином — и подожгли…
Когда дым рассеялся — мы заметили зверька,
сидящего в углу кровати
в позе Будды (ростом с 1/4 аршина)
И, как би-ба-бо ехидно улыбающегося.
Поняв, что это ОСОБОЕ существо,
я отправился за спиртом в аптеку
а тем временем
приятели ввертели ему окурками в живот
пепельницу.
Топтали каблуками, били по щекам, поджаривали уши,
а кто-то накаливал спинку кровати на свечке.
Вернувшись. я спросил:
— Ну как?
В темноте тихо ответили:
— Все уже кончено!
— Сожгли?
— Нет, сам застрелился…
ПОТОМУ ЧТО, сказал он,
В ОГНЕ Я УЗНАЛ НЕЧТО ЛУЧШЕЕ!


No one cares

Я прожарил свой мозг на железном пруте
Добавляя перцу румян и кислот
Чтоб